In [20]:
# 导入需要的库
import torch                       # PyTorch 主库
import torch.nn.functional as F    # 常用的神经网络函数库（激活函数、loss 等）
import matplotlib.pyplot as plt    # 画图用
%matplotlib inline                

In [21]:
# 读取所有名字数据，每一行是一个名字
words = open('names.txt', 'r').read().splitlines()
words[:8]  # 看前 8 个，确认读进来长什么样

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [22]:
len(words)  # 一共有多少个名字

32033

In [23]:
# 构建字符表：字符到整数、整数到字符的双向映射
chars = sorted(list(set(''.join(words))))     # 所有名字拼在一起，取去重后的所有字符
stoi = {s: i+1 for i, s in enumerate(chars)}  # 每个字符分配一个从 1 开始的 id
stoi['.'] = 0                                 # 特殊符号 '.' 用 0 表示（作为开始/结束符）
itos = {i: s for s, i in stoi.items()}        # 反向映射：id -> 字符
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [24]:
# 最简单版本：直接把所有 (context, next_char) 做成 X, Y
# context：前 block_size 个字符（用整数 id 表示）
# Y：下一个要预测的字符的 id

block_size = 3  # 上下文长度：用 3 个字符预测下一个
X, Y = [], []
for w in words:
  
  # print(w)  # 如果想看过程，可以打开
  context = [0] * block_size      # 用 0 作为起始上下文（相当于前面补 '...'）
  for ch in w + '.':              # 最后再拼一个 '.' 作为结束符
    ix = stoi[ch]                 # 当前字符的 id
    X.append(context)             # 把当前的 3 个字符（context）存进 X
    Y.append(ix)                  # 下一个要预测的字符 id 存进 Y
    # print(''.join(itos[i] for i in context), '--->', itos[ix])
    # 上面这一行可以打印：当前上下文字符 -> 目标字符
    context = context[1:] + [ix]  # 滑动窗口：丢掉最早的一个，加上当前的 id
  
X = torch.tensor(X)               # 转成张量，形状大概是 (样本数, 3)
Y = torch.tensor(Y)               # 一维张量，长度和 X 行数相同

In [25]:
X.shape, X.dtype, Y.shape, Y.dtype  # 看一下形状和数据类型

(torch.Size([228146, 3]), torch.int64, torch.Size([228146]), torch.int64)

In [26]:
# 更“正规”的版本：封装成函数，并划分 train / dev / test
block_size = 3  # 再次声明，方便下面引用

def build_dataset(words):
  """
  给定一组词（字符串列表），生成对应的 (X, Y) 数据集。
  X: 每一行是一个长度为 block_size 的上下文（由整数 id 组成）
  Y: 每一行是一个目标字符 id
  """
  X, Y = [], []
  for w in words:

    # print(w)
    context = [0] * block_size          # 初始上下文全 0
    for ch in w + '.':                  # 依次遍历名字里的字符，再加终止符 '.'
      ix = stoi[ch]                     # 字符 -> id
      X.append(context)                 # 当前上下文
      Y.append(ix)                      # 目标字符
      # print(''.join(itos[i] for i in context), '--->', itos[ix])
      context = context[1:] + [ix]      # 滑动窗口向前移动一格

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)               # 打印一下这个子数据集的大小
  return X, Y

import random
random.seed(42)                         # 固定随机种子，保证可复现
random.shuffle(words)                   # 打乱所有名字的顺序

n1 = int(0.8 * len(words))              # 前 80% 作为训练集
n2 = int(0.9 * len(words))              # 接下来 10% 作为 dev，最后 10% 作为 test

Xtr, Ytr = build_dataset(words[:n1])    # 训练集
Xdev, Ydev = build_dataset(words[n1:n2])# 验证集
Xte, Yte = build_dataset(words[n2:])    # 测试集



torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [27]:
# 创建embedding嵌入矩阵 C：把离散的字符 id 映射到一个低维向量空间
C = torch.randn((27, 2))  # 一共有 27 个符号，每个用 2 维向量表示（随机初始化）

In [28]:
# 使用嵌入矩阵查表：
# X 是 (N, block_size) 的整数张量，C[X] 会得到 (N, block_size, 2) 的嵌入向量
emb = C[X]
emb.shape  # (样本数, 3, 2)


torch.Size([228146, 3, 2])

In [29]:
# 定义第一层（隐藏层）的权重和偏置
# 输入维度 = 3（block_size）* 2（嵌入维度） = 6
# 隐藏层维度 = 100
W1 = torch.randn((6, 100))  # 权重矩阵
b1 = torch.randn(100)       # 偏置向量


In [30]:
# 把 (N, 3, 2) 展平为 (N, 6)，然后做线性变换 + tanh 激活
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)  # h: (N, 100)，隐藏层输出

In [31]:
h

tensor([[ 0.3348, -0.5060,  0.1968,  ..., -0.9916, -0.9821,  0.8410],
        [-0.4534, -0.4739,  0.9383,  ..., -0.9633,  0.1245,  0.9146],
        [ 0.9494,  0.8611, -0.9748,  ..., -0.9998, -0.9998,  0.1556],
        ...,
        [-0.9973, -0.8212,  0.9888,  ...,  0.9542, -0.9993, -0.8426],
        [-0.7395,  0.4131,  0.7871,  ...,  0.9734, -0.9940,  0.7995],
        [-0.9153, -0.5571,  0.9944,  ...,  0.9863, -0.9909, -0.8887]])

In [32]:
h.shape

torch.Size([228146, 100])

In [33]:
# 定义输出层参数：从 100 维隐藏层 -> 27 维输出（每个字符一个 logit）
W2 = torch.randn((100, 27))
b2 = torch.randn(27)


In [34]:
# 计算输出 logits（未归一化的分数）
logits = h @ W2 + b2  # 形状 (N, 27)

In [35]:
logits.shape

torch.Size([228146, 27])

In [36]:
# 手工计算 softmax（只做示意，后面会用 F.cross_entropy 更efficient）
counts = logits.exp()                          # 先对每个 logit 做指数
prob = counts / counts.sum(1, keepdims=True)   # 每一行除以这一行的和 -> 概率分布
prob.shape                                     # (N, 27)


torch.Size([228146, 27])

In [37]:
# 简单手写一个 loss（负对数似然），这里只是说明：
#loss = -prob[torch.arange(32), Y].log().mean()  # 这里只用了前 32 个样本
#loss


In [38]:
# ------------ now made respectable :) ---------------

In [39]:
# 正式开始用 train/dev/test + MLP + cross_entropy 训练

Xtr.shape, Ytr.shape  # 看训练集尺寸（样本数，3），（样本数,）

(torch.Size([182625, 3]), torch.Size([182625]))

In [40]:
# 用固定随机种子初始化所有参数
g = torch.Generator().manual_seed(2147483647)   # 为 PyTorch 的随机数生成器设定种子
C = torch.randn((27, 10),  generator=g)         # 嵌入维度改为 10
W1 = torch.randn((30, 200), generator=g)        # 输入 3*10=30，隐藏层 200
b1 = torch.randn(200,      generator=g)
W2 = torch.randn((200, 27), generator=g)        # 从 200 -> 27
b2 = torch.randn(27,        generator=g)
parameters = [C, W1, b1, W2, b2]                # 把所有参数放在一个列表里方便操作


In [41]:
# 统计一下总参数量
sum(p.nelement() for p in parameters)  # 所有参数的元素个数总和

11897

In [42]:
# 开启所有参数的梯度跟踪
for p in parameters:
  p.requires_grad = True

In [43]:
# 可选：准备一组学习率，用于搜索（这里只是先生成，下面注释掉了）
lre = torch.linspace(-3, 0, 1000)  # 在 [10^-3, 10^0] 上取对数均匀的 1000 个点
lrs = 10**lre                       # 实际的学习率数组（指数变换回来）


In [44]:
# 用来记录训练过程的学习率、loss、步数
lri = []       # 对应 lre 的索引（这里实际没用到）
lossi = []     # 每一步的 loss（取 log10 方便画图）
stepi = []     # 训练步数 i

In [45]:
# 训练循环：在训练集上随机抽取 minibatch 做 SGD
for i in range(200000):
  
  # minibatch 构造：随机取 32 个样本
  ix = torch.randint(0, Xtr.shape[0], (32,))
  
  # forward pass
  emb = C[Xtr[ix]]                             # (32, 3, 10)，查嵌入
  h = torch.tanh(emb.view(-1, 30) @ W1 + b1)   # (32, 200) 隐藏层
  logits = h @ W2 + b2                         # (32, 27) 输出层
  loss = F.cross_entropy(logits, Ytr[ix])      # 直接用内置的 cross_entropy 计算 NLL loss
  # print(loss.item())
  
  # backward pass：梯度清零 + 反向传播
  for p in parameters:
    p.grad = None
  loss.backward()
  
  # 参数更新（带简单的学习率 schedule）
  # lr = lrs[i]               # 如果想用前面准备的学习率表，可以改用这行
  lr = 0.1 if i < 100000 else 0.01  # 前 10 万步用 0.1，后 10 万步用 0.01
  for p in parameters:
    p.data += -lr * p.grad          # 梯度下降更新参数

  # 记录训练过程的统计
  # lri.append(lre[i])
  stepi.append(i)
  lossi.append(loss.log10().item()) # 用 log10(loss) 方便可视化范围

# print(loss.item())  # 最后看一下最终 loss


In [46]:
# 画出训练过程中 loss 的变化
plt.plot(stepi, lossi)

In [47]:
# 在训练集上计算最终损失（使用所有训练样本）
emb = C[Xtr]                               # (Ntr, 3, 10)
h = torch.tanh(emb.view(-1, 30) @ W1 + b1) # (Ntr, 200)
logits = h @ W2 + b2                       # (Ntr, 27)
loss = F.cross_entropy(logits, Ytr)        # 训练集 loss
loss

tensor(2.1136, grad_fn=<NllLossBackward0>)

In [48]:
# 在 dev 集上计算损失，用于评估泛化效果
emb = C[Xdev]                              # (Ndev, 3, 10)
h = torch.tanh(emb.view(-1, 30) @ W1 + b1) # (Ndev, 200)
logits = h @ W2 + b2                       # (Ndev, 27)
loss = F.cross_entropy(logits, Ydev)       # 验证集 loss
loss

tensor(2.1540, grad_fn=<NllLossBackward0>)

In [49]:
# 可视化嵌入矩阵 C 的前两个维度（只是一个 low-dim 可视化）
# 看不同字符在嵌入空间中的分布
plt.figure(figsize=(8, 8))
plt.scatter(C[:, 0].data, C[:, 1].data, s=200)
for i in range(C.shape[0]):
    plt.text(C[i, 0].item(), C[i, 1].item(), itos[i],
             ha="center", va="center", color='white')
plt.grid('minor')

In [50]:
# 这里是关于 train / dev / test 划分的小注释：
# training split, dev/validation split, test split
# 80%, 10%, 10%

In [51]:
context = [0] * block_size
C[torch.tensor([context])].shape

torch.Size([1, 3, 10])

In [52]:
# 从训练好的模型中采样生成名字
g = torch.Generator().manual_seed(2147483647 + 10)  # 新的随机种子，防止和训练阶段冲突

for _ in range(20):  # 生成 20 个名字
    
    out = []                                   # 用来存储生成的字符 id 序列
    context = [0] * block_size                 # 初始上下文全 0，相当于 '...'
    while True:
      emb = C[torch.tensor([context])]         # (1, block_size, 10)
      h = torch.tanh(emb.view(1, -1) @ W1 + b1)# (1, 200)
      logits = h @ W2 + b2                     # (1, 27)
      probs = F.softmax(logits, dim=1)         # softmax -> 概率分布
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
                                               # 按概率分布随机采样一个字符 id
      context = context[1:] + [ix]             # 更新上下文（滑动窗口）
      out.append(ix)                           # 记录当前采样的字符 id
      if ix == 0:                              # 如果采到 0（即 '.'），结束这个名字
        break
    
    print(''.join(itos[i] for i in out))       # 把 id 序列转成字符串并打印出来








carmahxa.
jehmarik.
mis.
reh.
caspanden.
jazhett.
deliah.
jarqui.
ner.
kia.
chaiir.
kaleigh.
ham.
jord.
quint.
shoisea.
jaddi.
wazelo.
dearyxia.
kael.
